# read  unredshifted Fors2 spectra from H5 file

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- creation date : 2023/05/23
- last update : 2023/05/23
- FORS2 : https://github.com/ericnuss/fors2


- MUST RUN *convertFors2spectratohdf5.ipynb* before

# Import

In [ ]:
import os,sys,re
import h5py
from astropy.table import Table
import random

In [ ]:
# Import some generally useful packages

import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

from matplotlib.backends.backend_pdf import PdfPages

%matplotlib inline
import pandas as pd
from itertools import cycle, islice

In [ ]:
from scipy import ndimage

#from scipy import interp as scinterp
from scipy.interpolate import interp1d

import glob
import collections
#from def_raw_seds import *

In [ ]:
# to enlarge the sizes
params = {'legend.fontsize': 'large',
          'figure.figsize': (4, 4),
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}
plt.rcParams.update(params)

In [ ]:
from GalaxyLineDict import build_emissionlinesdict

In [ ]:
def plot_line(ax,line_name):
    
    #keys = lines_to_plot.keys()
    #for the_key in keys:
    #    if the_key in line_name:
    #        print("the_key=",the_key," line_name=",line_name)
    #        line_name=the_key
    #        break
            
    posx0=0.0
    posy0=0.05
    fontsize=12
    the_color = {"emission":'r',"absorption":"g","break":"purple"}
    the_line_to_plot = lines_to_plot[line_name]
    wls = the_line_to_plot["wls"]
    tag = the_line_to_plot["name"]
    typ = the_line_to_plot["type"]
    
    xlim = ax.get_xlim()
    for wl in wls:
        ax.axvline(wl, lw=2, color=the_color[typ])
        xpos = (wl - xlim[0]) / (xlim[1] - xlim[0])+posx0 
        if 0 < xpos < 1:
            ax.annotate(tag, xy=(xpos, posy0), rotation=0, ha='left', va='bottom',
                            xycoords='axes fraction', color=the_color[typ], fontsize=fontsize)
    return ax

In [ ]:
lines_to_plot=build_emissionlinesdict()

# Config

In [ ]:
input_file_h5  = 'FORS2spectra.hdf5'

In [ ]:
hf =  h5py.File(input_file_h5, 'r') 
list_of_keys = list(hf.keys())

In [ ]:
print(list_of_keys)

In [ ]:
# pick one key    
key_sel =  list_of_keys[0]
# pick one group
group = hf.get(key_sel)  

In [ ]:
#pickup all attribute names
all_subgroup_keys = []
for k in group.attrs.keys():
    all_subgroup_keys.append(k)

In [ ]:
all_subgroup_keys

In [ ]:
def GetColumnHfData(hff,list_of_keys,nameval):
    """
    Extract hff atttribute 
    
    parameters
      hff           : descriptor of h5 file
      list_of_keys : list of exposures
      nameval      : name of the attribute
      
    return
           the array of values in the order of 
    """
    

    all_data = []
    for key in list_of_keys:
        group=hff.get(key)
        val=group.attrs[nameval]
        all_data.append(val)
    return all_data


In [ ]:
# create info
df_info = pd.DataFrame()
for key in all_subgroup_keys:
    arr=GetColumnHfData(hf, list_of_keys ,key)
    df_info[key] = arr

In [ ]:
df_info.sort_values(by="num", ascending=True,inplace=True)

In [ ]:
df_info_num = df_info["num"].values

In [ ]:
key_tags = [ f"SPEC{num}" for num in df_info_num ]

In [ ]:
all_df = []  
    
idx=0
for key in key_tags :
        
    group = hf.get(key)
               
    df = pd.DataFrame()
    
        
    df["wl"] = np.array(group.get("wl"))
    df["fl"] = np.array(group.get("fl")) 
 
    all_df.append(df)
        
    idx+=1

In [ ]:
N = len(all_df)
fig = plt.figure(figsize=(10,6))
ax=fig.add_subplot(1,1,1)
for idx in range(N):
    ax.plot(all_df[idx]["wl"], all_df[idx]["fl"])
ax.set_ylim(0,30)    

In [ ]:
fig = plt.figure(figsize=(10,6))
ax=fig.add_subplot(1,1,1)
ax.plot(all_df[0]["wl"], all_df[0]["fl"])

In [ ]:
df_info

In [ ]:
NSED = len(df_info)

# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=0, vmax=NSED)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
df_info.iloc[1]["num"]

In [ ]:
for idx in range(0,NSED):
    row = df_info.iloc[idx]
    num= row["num"]
    the_tag=f"SPEC{num}"
    print(f"===================== SED {idx} : {the_tag}=======================")
    
    the_lines = row["lines"]
    print("the_lines = ",the_lines)
    all_elements  = the_lines.decode().split(",")
    #fig, ax = plt.subplot(1,1,1,figsize=(16,8))
    #fig, axs = plt.subplots(1, 1, figsize=(16, 8),dpi=50)
    fig, axs = plt.subplots(1, 1, figsize=(16, 8))
    ax=axs
    ax.plot(all_df[idx]["wl"], all_df[idx]["fl"],'b-' ,color=all_colors[idx],label=the_tag,lw=3)
    ax.legend(loc="upper right")
    ax.grid()
    #ax.axvline(4150,color="grey")
    #ax.axvline(4250,color="grey")
#    plt.ylim(-0.02,0.4)
    ax.set_xlim(4000.,10000.)
    for el in all_elements:
        el=el.split(" ")[0]  
        el=el.split("\r")[0]  
        print(el)
        if len(el)>0:
            plot_line(ax,el)
    plt.show()
    del fig